In [10]:
import pandas as pd
import numpy as np

In [11]:
path = "../data/cleaned_data/top_100_startup_2025.csv"
df = pd.read_csv(path)

In [12]:
df.head()

,nom_entreprise,annee_creation,QG,secteur,nombre_employes,top_investisseurs,produit,cote_g2,capital_$,ca_$,valorisation_$
0,Microsoft,1975,"Redmond, WA, USA",Enterprise Software,221000.0,"Bill Gates, Paul Allen","Azure, Office 365, Teams",4.4,1.000000e+09,2.700000e+11,3.000000e+12
1,Salesforce,1999,"San Francisco, CA, USA",CRM,75000.0,"Halsey Minor, Larry Ellison","Sales Cloud, Service Cloud",4.3,6.540000e+08,3.790000e+10,2.278000e+11
2,Adobe,1982,"San Jose, CA, USA",Creative Software,29945.0,Hambrecht & Quist,"Creative Cloud, Document Cloud",4.5,2.500000e+07,1.940000e+10,2.400000e+11
3,Oracle,1977,"Austin, TX, USA",Database & Enterprise,143000.0,"Larry Ellison, Bob Miner","Oracle Cloud, NetSuite",4.0,2.000000e+00,5.290000e+10,3.500000e+11
4,Intuit,1983,"Mountain View, CA, USA",Financial Software,18200.0,"Sierra Ventures, Kleiner Perkins","QuickBooks, TurboTax",4.4,2.730000e+08,1.440000e+10,1.800000e+11


# feature engineering

Ajout de colonnes: <br>
1- Cote d’activité qui représente le niveau d’activité de l’entreprise <br>
2- Cote de crédit (ou « score de crédit ») qui représente la solvabilité financière de l'entreprise

In [21]:
colonnes = ["nombre_employes", "capital_$", "ca_$", "valorisation_$", "cote_g2"]
df_final = df[colonnes].copy()

In [22]:

# Critères sur le CA pour calculer la cote d'activité
g2_cotation_to_turnover_usd = {
    "A": [810_000_000, float("inf")],       # ≥ 750 M€
    "B": [162_000_000, 810_000_000],        # 150–750 M€
    "C": [54_000_000, 162_000_000],         # 50–150 M€
    "D": [32_400_000, 54_000_000],          # 30–50 M€
    "E": [16_200_000, 32_400_000],          # 15–30 M€
    "F": [8_100_000, 16_200_000],           # 7.5–15 M€
    "G": [1_620_000, 8_100_000],            # 1.5–7.5 M€
    "H": [810_000, 1_620_000],              # 0.75–1.5 M€
    "J": [540_000, 810_000],                # 0.5–0.75 M€
    "K": [270_000, 540_000],                # 0.25–0.5 M€
    "L": [108_000, 270_000],                # 0.10–0.25 M€
    "M": [0, 108_000],                      # < 0.10 M€
    "N": None,                              # Non significatif
    "X": None                               # Inconnu ou trop ancien
}

colonne = []

for v in df_features ["ca_$"]:
    if v >= 810_000_000:
        colonne.append("A")
    elif v >= 162_000_000:
        colonne.append("B")
    elif v >= 54_000_000:
        colonne.append("C")
    elif v >= 32_400_000:
        colonne.append("D")
    elif v >= 16_200_000:
        colonne.append("E")
    elif v >= 8_100_000:
        colonne.append("F")
    elif v >= 1_620_000:
        colonne.append("G")
    elif v >= 810_000:
        colonne.append("H")
    elif v >= 540_000:
        colonne.append("J")
    elif v >= 270_000:
        colonne.append("K")
    elif v >= 108_000:
        colonne.append("L")
    elif v >= 0:
        colonne.append("M")
    else:
        colonne.append("X")  # Valeur manquante ou négative

df_final["cote_activite"] = colonne

In [23]:
# Ajoute de la colonne "cote_cativite"
freq_cotes = df_final["cote_activite"].value_counts(normalize=True)
df_final["cote_activite"] =df_final["cote_activite"].map(freq_cotes)

Critère pour la cote de crédit:
| Critère                                   | Pondération approximative |
| ----------------------------------------- | ------------------------- |
| Historique de paiements                   | 35 %                      |
| Montant dû / taux d’utilisation du crédit | 30 %                      |
| Longévité de l’historique                 | 15 %                      |
| Type de crédit utilisé                    | 10 %                      |
| Nouvelles demandes de crédit              | 10 %                      |


In [16]:
# exemple d'un modèle de calcul
def credit_score(ebitda_margin, debt_equity_ratio, ca_growth, payment_history_score):
    score = (
        ebitda_margin * 30 +
        (1 / (1 + debt_equity_ratio)) * 25 +
        ca_growth * 20 +
        payment_history_score * 25
    )
    return round(score, 2)

Pas assez d'éléments pour ajouter cette colonne, pour l'instant !!! 

In [27]:
df_final.to_csv("../data/final_data/top_100_startup_2025.csv")